In [ ]:
# Standard Libraries
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Libraries for Images 
import os
import glob
from PIL import Image

# Libraries for Evaluations and Train/Test Split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Model Libraries
from kerastuner.tuners import BayesianOptimization
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np



from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from kerastuner.tuners import BayesianOptimization
from tensorflow.keras.models import load_model

from kerastuner.tuners import BayesianOptimization
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

#VGG16
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam


#ResNET
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

#Statistical Test
from scipy.stats import ttest_rel
from sklearn.metrics import accuracy_score

#Evaluation
from sklearn.metrics import accuracy_score
from scipy.stats import ttest_rel
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

#Model Visualization
from ann_visualizer.visualize import ann_viz
from tensorflow.keras.utils import plot_model


#Others
from tqdm import tqdm  # Optional: tqdm for progress bar
from concurrent.futures import ThreadPoolExecutor

## CNN2 - max pooling, relu, batch normalization, dropout

In [ ]:
model_cnn2 = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    
    Flatten(),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    
    Flatten(),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    
    Dense(num_classes, activation='softmax')
])

model_cnn2.compile(optimizer=tf.keras.optimizers.legacy.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
model_cnn2.summary()

    # Define a callback to save the best model during training
checkpoint = ModelCheckpoint("best_model_cnn2.h5", monitor='val_loss', save_best_only=True)
    # Define a callback to save the best model during training in the native Keras format
#checkpoint = ModelCheckpoint("best_model.keras", monitor='val_loss', save_best_only=True)


# Train the model
history = model_cnn2.fit(
    train_generator,
    epochs=10,  # Adjust the number of epochs based on your needs
    validation_data=validation_generator,
    callbacks=[checkpoint]
)


In [1]:
import tensorflow as tf
import keras

print("TensorFlow version:", tf.__version__)
print("Keras version:", keras.__version__)

TensorFlow version: 2.15.0
Keras version: 3.0.4


In [ ]:
def k_fold_cross_validation(original_model, train_generator, n_splits=5, epochs=5):
    dataset_size = train_generator.n
    num_classes = len(train_generator.class_indices)
    
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

    fold_performance = []

    for train_indices, val_indices in kf.split(range(dataset_size)):
        # Creating train and validation sub-generators for each fold
        train_subgen = create_subgenerator(train_generator, train_indices)
        val_subgen = create_subgenerator(train_generator, val_indices)

        # Cloning the original model structure
        model = clone_model(original_model)
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

        # Train the model
        history = model.fit(train_subgen, epochs=epochs, validation_data=val_subgen, steps_per_epoch=len(train_indices)//train_generator.batch_size, validation_steps=len(val_indices)//train_generator.batch_size)

        # Evaluate the model
        scores = model.evaluate(val_subgen, verbose=0, steps=len(val_indices)//train_generator.batch_size)
        fold_performance.append(scores[1])  # Assuming [1] is accuracy

    # Average performance
    average_performance = np.mean(fold_performance)
    return fold_performance, average_performance

## CNN3 - max pooling, stride 1,2 , padding = valid, relu

In [ ]:
# Define the CNN model
#num_classes = len(train_generator.class_indices)

model_cnn3 = tf.keras.models.Sequential([
    Conv2D(32 , (3,3) , strides = 1 , padding = 'valid' , activation = 'relu', input_shape=(256, 256, 3)),
    MaxPooling2D((2,2) , strides = 2 , padding = 'valid'),

    Conv2D(64 , (3,3) , strides = 1 , padding = 'valid' , activation = 'relu'),
    MaxPooling2D((2,2) , strides = 2 , padding = 'valid'),
    
    Conv2D(64 , (3,3) , strides = 1 , padding = 'valid' , activation = 'relu'),
    MaxPooling2D((2,2) , strides = 2 , padding = 'valid'),
    
    Conv2D(128 , (3,3) , strides = 1 , padding = 'valid' , activation = 'relu'),
    MaxPooling2D((2,2) , strides = 2 , padding = 'valid'),
    
    Flatten(),
    Dense(units = 128 , activation = 'relu'),
    Dropout(0.2),
    Dense(units = num_classes, activation='softmax')  # Adjust 'num_classes' based on your task
])


# Compile the model
#model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Compile the model with the legacy Adam optimizer
model_cnn3.compile(optimizer=tf.keras.optimizers.legacy.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])


# Display the model summary
model_cnn3.summary()

    # Define a callback to save the best model during training
checkpoint = ModelCheckpoint("best_model_cnn3.h5", monitor='val_loss', save_best_only=True)
    # Define a callback to save the best model during training in the native Keras format
#checkpoint = ModelCheckpoint("best_model.keras", monitor='val_loss', save_best_only=True)


# Train the model
history = model_cnn3.fit(
    train_generator,
    epochs=100,  # Adjust the number of epochs based on your needs
    validation_data=validation_generator,
    callbacks=[checkpoint]
)

In [ ]:
def k_fold_cross_validation(original_model, train_generator, n_splits=5, epochs=5):
    # Extracting data and labels from the generator
    dataset_size = train_generator.n
    num_classes = len(train_generator.class_indices)
    batch_size = train_generator.batch_size
    target_size = train_generator.target_size

    X = np.zeros((dataset_size, target_size[0], target_size[1], 3), dtype=np.float32)
    y = np.zeros((dataset_size, num_classes), dtype=np.float32)

    for i in range(len(train_generator)):
        batch_x, batch_y = train_generator.next()
        X[i * batch_size:(i + 1) * batch_size] = batch_x
        y[i * batch_size:(i + 1) * batch_size] = batch_y

    # K-Fold Cross-Validation
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    fold_performance = []

    for train_indices, val_indices in kf.split(X):
        # Splitting data
        X_train, X_val = X[train_indices], X[val_indices]
        y_train, y_val = y[train_indices], y[val_indices]

        # Cloning the original model structure
        model = clone_model(original_model)
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

        # Training the model
        model.fit(X_train, y_train, epochs=epochs, validation_data=(X_val, y_val))

        # Evaluating the model
        scores = model.evaluate(X_val, y_val, verbose=0)
        fold_performance.append(scores[1])  # Assuming [1] is accuracy

    # Average performance
    average_performance = np.mean(fold_performance)
    return fold_performance, average_performance